# Download other datasets

- [rogersdepelle/OffComBR: Here we provide a data set of web comments which have been annotated for hate speech.](https://github.com/rogersdepelle/OffComBR)
- [LaCAfe/Dataset-Hatespeech: Hate Speech Detection Dataset](https://github.com/LaCAfe/Dataset-Hatespeech)
- [paulafortuna/Portuguese-Hate-Speech-Dataset: A Hierarchically-Labeled Portuguese Hate Speech Dataset](https://github.com/paulafortuna/Portuguese-Hate-Speech-Dataset)
- [JAugusto97/ToLD-Br: Toxic Language Detection in Social Media for Brazilian Portuguese: New Dataset and Multilingual Analysis](https://github.com/JAugusto97/ToLD-Br)

In [2]:
import sys
from pathlib import Path

if str(Path(".").absolute().parent) not in sys.path:
    sys.path.append(str(Path(".").absolute().parent.parent))

In [7]:
import io
import os
import datetime
import requests
import pandas as pd
from tqdm import tqdm
from src.anonymize import Anonymizer
from src.data_models import RawText
from src.perspective import PerspectiveAPI
from src.settings import AppSettings
from src.s3 import Uploader
from src.utils import label_studio_fmt
from typing import Dict, List, Any
from dotenv import load_dotenv

# Initialize the env vars
load_dotenv("../../.env")

args = AppSettings()

args.AWS_S3_BUCKET_PREFIX += "/other_datasets"

In [3]:
def get_file_github(url: str, extension: str = "csv", pd_args: Dict[Any, Any] = {}) -> List[Any]:
    """
    Download a file from a github url

    Args:
    - url: the url to the file
    - extension: the extension of the file

    Returns:
    - pd.DataFrame: the dataframe of the file
    """
    download = requests.get(url).content
    
    if "csv" in extension.lower():
        df = pd.read_csv(io.StringIO(download.decode('utf-8')), **pd_args)
    elif "arff" in extension.lower():
        df = pd.read_csv(io.StringIO(download.decode('utf-8')), **pd_args)
    elif "json" in extension.lower():
        df = pd.read_json(io.StringIO(download.decode('utf-8')), **pd_args)
    else:
        raise ValueError("Only csv, arff and json are supported")
    return df.to_dict(orient="records")

perspective = PerspectiveAPI(apikey=os.getenv("PERSPECTIVE_API_KEY"))

anonymizer = Anonymizer()

### rogersdepelle/OffComBR

In [4]:
url = "https://raw.githubusercontent.com/rogersdepelle/OffComBR/master/OffComBR2.arff"

offcombr = get_file_github(url, extension="arff")
offcombr = offcombr[3:]

print(f"Count: {len(offcombr)}")
offcombr

Count: 1250


[{'@relation data': "'Votaram no PEZAO Agora tomem no CZAO'"},
 {'@relation data': "'cuidado com a poupanca pessoal Lembram o que aconteceu na epoca do Collor ne'"},
 {'@relation data': "'Sabe o que eu acho engracado os nossos governantes  nao pensam em cortar regalias e beneficios desnecessarios que os favorecem porque sera ne e mais do que claro ate mesmo para quem nao quer enxergar eles sao estao la para defender seus proprios interesses  e os dos empresario o no casso eles tambem ou comecamos a tomar uma atitude para mudar de uma vez por todas essa roubalheira nesse pais o a tendencia e so piorar para o povo porque dinheiro para investimentos nao tem mais para aumentos de salario e regalias nao falta'"},
 {'@relation data': "'os cariocas tem o que merecem um pessoal que so sabem toma banho de sol e pratica a violencia e nao deu outra de onde se tira e nao coloca um dia acaba'"},
 {'@relation data': "'Podiam retirar dos lucros dos bancos '"},
 {'@relation data': "'CADE O GALVAO PRA 

In [10]:
lst = []

with tqdm(total=len(offcombr)) as pbar:
    for row in offcombr:
        if len(row["@relation data"]) < 500:
            item = RawText(
                id=datetime.datetime.now().timestamp().as_integer_ratio()[0],
                text=row["@relation data"],
                source="OffComBR",
                created_at=datetime.datetime(2017, 5, 8, 14, 50, 0),
                collected_at=datetime.datetime.now()
            )

            response = perspective.predict(text=item.text)

            if isinstance(response.get("TOXICITY"), float):
                if response.get("TOXICITY") > args.PERSPECTIVE_THRESHOLD:
                    item.is_toxic = True
                    item.toxicity_score = response.get("TOXICITY")

            item.text = anonymizer.apply_all(item.text)
            
            if item.is_toxic or not args.FILTER_TOXIC_COMMENTS:
                lst.append(item)
            
        pbar.update(1)

lst = [label_studio_fmt(i) for i in lst]


100%|██████████| 1250/1250 [06:51<00:00,  3.04it/s]


In [11]:
lst[0]

{'data': {'text': "'Votaram no PEZAO Agora tomem no CZAO'",
  'ref_id': 6888588279223553,
  'meta_info': {'source': 'OffComBR',
   'created_at': '2017-05-08T14:50:00',
   'collected_at': '2022-01-16T18:10:29.548157',
   'is_toxic': True,
   'toxicity_score': 0.5424}}}

In [12]:
uploader = Uploader(bucket=args.AWS_S3_BUCKET,
                    bucket_prefix=args.AWS_S3_BUCKET_PREFIX)

uploader.upload_aksk(access_key=args.AWS_ACCESS_KEY_ID,
                     secret_key=args.AWS_SECRET_ACCESS_KEY,
                     key="offcombr.json", data=lst)

{'ResponseMetadata': {'RequestId': 'S8YEHFJDPHG5NRHR',
  'HostId': 'vNYYnYqMmF00zhQiJmbhM/k54TBZ6nwoIn/URch4dJ/PVC/NWv9OVrlL0eUDL5+JNrwMm1XX004=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'vNYYnYqMmF00zhQiJmbhM/k54TBZ6nwoIn/URch4dJ/PVC/NWv9OVrlL0eUDL5+JNrwMm1XX004=',
   'x-amz-request-id': 'S8YEHFJDPHG5NRHR',
   'date': 'Sun, 16 Jan 2022 21:17:57 GMT',
   'x-amz-version-id': 'fFyo5tvnlYs1jquqH3KK1.VjPZ5.yVJs',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"64bca8b654e7101b1108324c4bf29de1"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"64bca8b654e7101b1108324c4bf29de1"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'fFyo5tvnlYs1jquqH3KK1.VjPZ5.yVJs'}

### LaCAfe/Dataset-Hatespeech

In [13]:
url = "https://raw.githubusercontent.com/LaCAfe/Dataset-Hatespeech/master/data/df_dataset.json"

nccvg = get_file_github(url, extension="json")
print(f"Count: {len(nccvg)}")
nccvg

Count: 7672


[{'docno': 23310,
  'has_anger': 'S',
  'origin': '55chan/pol',
  'txt': '>>22994apóio o >>22995. um passo de cada vez. não tenha pressa, mas sempre movendo pra frente. e não tenha medo de pedir ajuda a amigos, parentes ou psicólogos. não tem nada de mais.e outra: faculdade nem sempre é a solução. tem cursos técnicos que também contam como superior e que às vezes são mais curtos que faculdades e te pagam melhor.'},
 {'docno': 1141756984838053888,
  'has_anger': None,
  'origin': 'twitter',
  'txt': 'eu ainda vou surtar com essa fic'},
 {'docno': 18700,
  'has_anger': 'S',
  'origin': '55chan/pol',
  'txt': '>>11826 puta merda… se alguém de fato analisar esse chart…. saberá o motivo que o mundo ta na merda.'},
 {'docno': 63576,
  'has_anger': 'S',
  'origin': '55chan/pol',
  'txt': 'seguinte, fizemos 4 anos de namoro semana passada, mas nós estávamos brigados e não comemoramos nada. fiquei sabendo que, no dia do nosso aniversário de namoro, ela tinha saído com um amigo. não dei a miníma

In [20]:
lst = []

with tqdm(total=len(nccvg)) as pbar:
    for row in nccvg:
        if len(row["txt"]) < 500:
            item = RawText(
                id=datetime.datetime.now().timestamp().as_integer_ratio()[0],
                text=row["txt"],
                source="NCCVG",
                created_at=datetime.datetime(2019, 9, 7, 15, 14, 0),
                collected_at=datetime.datetime.now()
            )

            response = perspective.predict(text=item.text)

            if isinstance(response.get("TOXICITY"), float):
                if response.get("TOXICITY") > args.PERSPECTIVE_THRESHOLD:
                    item.is_toxic = True
                    item.toxicity_score = response.get("TOXICITY")

            item.text = anonymizer.apply_all(item.text)
            
            if item.is_toxic or not args.FILTER_TOXIC_COMMENTS:
                lst.append(item)
                
        pbar.update(1)

lst = [label_studio_fmt(i) for i in lst]

100%|██████████| 7672/7672 [33:03<00:00,  3.87it/s]  


In [21]:
uploader = Uploader(bucket=args.AWS_S3_BUCKET,
                    bucket_prefix=args.AWS_S3_BUCKET_PREFIX)

uploader.upload_aksk(access_key=args.AWS_ACCESS_KEY_ID,
                     secret_key=args.AWS_SECRET_ACCESS_KEY,
                     key="nccvg.json", data=lst)

{'ResponseMetadata': {'RequestId': '7P38HYH4D6CQ1SRG',
  'HostId': 'yTdQ5r/LhLz5XEcn2NxP9eQ8Gjc+ut9EnOKmNusGGeAZgXD+3CpthWmoUXQ4x4HdPwsyaddAtR8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'yTdQ5r/LhLz5XEcn2NxP9eQ8Gjc+ut9EnOKmNusGGeAZgXD+3CpthWmoUXQ4x4HdPwsyaddAtR8=',
   'x-amz-request-id': '7P38HYH4D6CQ1SRG',
   'date': 'Sun, 16 Jan 2022 22:00:02 GMT',
   'x-amz-version-id': '.qS1j._gPIE3RPIi5iB2XhDW1RV_UeTY',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"70d2b6f50254eaa5e6ab64a21e2c083b"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"70d2b6f50254eaa5e6ab64a21e2c083b"',
 'ServerSideEncryption': 'AES256',
 'VersionId': '.qS1j._gPIE3RPIi5iB2XhDW1RV_UeTY'}

### paulafortuna/Portuguese-Hate-Speech-Dataset

In [22]:
url = "https://raw.githubusercontent.com/paulafortuna/Portuguese-Hate-Speech-Dataset/master/2019-05-28_portuguese_hate_speech_binary_classification.csv"

hlphsd = get_file_github(url, extension="csv")
print(f"Count: {len(hlphsd)}")
hlphsd

Count: 5670


[{'text': '@__andrea__b \nO cara vive em outro mundo\nNão no mundo real\nREFUGIADOS são os que vivem\nNas favelas vizinhas as suas fortalezas',
  'hatespeech_comb': 1,
  'hatespeech_G1': 1,
  'annotator_G1': 'A',
  'hatespeech_G2': 1.0,
  'annotator_G2': 'V',
  'hatespeech_G3': 0,
  'annotator_G3': 'E'},
 {'text': '@_carmeloneto Estes incompetentes não cuidam nem do povo brasileiro e nem dos poucos refugiados que aqui estão.',
  'hatespeech_comb': 0,
  'hatespeech_G1': 1,
  'annotator_G1': 'D',
  'hatespeech_G2': 0.0,
  'annotator_G2': 'V',
  'hatespeech_G3': 0,
  'annotator_G3': 'C'},
 {'text': "@_carmeloneto \nOs 'cumpanhero' quebraram todas as regras.",
  'hatespeech_comb': 0,
  'hatespeech_G1': 1,
  'annotator_G1': 'A',
  'hatespeech_G2': 0.0,
  'annotator_G2': 'B',
  'hatespeech_G3': 0,
  'annotator_G3': 'E'},
 {'text': '@_GlitteryKisses é isso não conseguem pensar no sentido lato para além do que se vê a frente dos olhos',
  'hatespeech_comb': 0,
  'hatespeech_G1': 0,
  'annotato

In [23]:
url = "https://raw.githubusercontent.com/paulafortuna/Portuguese-Hate-Speech-Dataset/master/2019-05-28_portuguese_hate_speech_hierarchical_classification.csv"

hlphsd2 = get_file_github(url, extension="csv")
print(f"Count: {len(hlphsd2)}")
hlphsd2

Count: 5668


[{'text': '"não come mel, morde marimbondo"',
  'Hate.speech': 0,
  'Sexism': 0,
  'Body': 0,
  'Racism': 0,
  'Ideology': 0,
  'Homophobia': 0,
  'Origin': 0,
  'Religion': 0,
  'Health': 0,
  'OtherLifestyle': 0,
  'Aborting.women': 0,
  'Agnostic': 0,
  'Argentines': 0,
  'Asians': 0,
  'Autists': 0,
  'Black.Women': 0,
  'Blond.women': 0,
  'Brazilians.women': 0,
  'Chinese': 0,
  'Criminals': 0,
  'Egyptians': 0,
  'Fat.people': 0,
  'Football.players.women': 0,
  'Gamers': 0,
  'Homeless': 0,
  'Homeless.women': 0,
  'Indigenous': 0,
  'Iranians': 0,
  'Japaneses': 0,
  'Jews': 0,
  'Jornalists': 0,
  'Latins': 0,
  'Left.wing.ideology': 0,
  'Men.Feminists': 0,
  'Mexicans': 0,
  'Muslims.women': 0,
  'Nordestines': 0,
  'Old.people': 0,
  'Polyamorous': 0,
  'Poor.people': 0,
  'Rural.people': 0,
  'Russians': 0,
  'Sertanejos': 0,
  'Street.artist': 0,
  'Ucranians': 0,
  'Vegetarians': 0,
  'White.people': 0,
  'Young.people': 0,
  'Old.women': 0,
  'Ugly.people': 0,
  'Venez

In [24]:
hlphsd = [row["text"] for row in hlphsd]

for row in hlphsd2:
    hlphsd.append(row["text"])

hlphsd = list(set(hlphsd))
print(f"Count: {len(hlphsd)}")
hlphsd[:5]

Count: 6417


["Amigos!!! Acabei de fazer estes 'terrários' para minha cunhada Vanessa Alves Scarpim.o que acharam? Que pena que... https://t.co/TMCsGufo5v",
 "O arranque da Tour 'Overdose de Tourette -OTR' foi assim.\nA caravana segue já dia 10 para Vila Real e dia 11 para a _ https://t.co/vkP9WZ5Sa8",
 "Deve ser essa a tal 'guinada à direita' que a imprensa tanto fala... \n#piada #sótemsocialistanessamerda https://t.co/fsUQ4K9Xi3",
 '@fofocasonline e com as crianças eles fazem isso...povo maldito mesmo ! @RachelSherazade https://t.co/GA0Ql2TaYn',
 '@GoncaloMirandaa \nNão se pode ter só "defeitos" ;)']

In [25]:
lst = []

with tqdm(total=len(hlphsd)) as pbar:
    for row in hlphsd:
        if len(row) < 500:
            item = RawText(
                id=datetime.datetime.now().timestamp().as_integer_ratio()[0],
                text=row,
                source="HLPHSD",
                created_at=datetime.datetime(2019, 5, 28, 13, 22, 0),
                collected_at=datetime.datetime.now()
            )

            response = perspective.predict(text=item.text)

            if isinstance(response.get("TOXICITY"), float):
                if response.get("TOXICITY") > args.PERSPECTIVE_THRESHOLD:
                    item.is_toxic = True
                    item.toxicity_score = response.get("TOXICITY")

            item.text = anonymizer.apply_all(item.text)
            
            if item.is_toxic or not args.FILTER_TOXIC_COMMENTS:
                lst.append(item)
                
        pbar.update(1)

lst = [label_studio_fmt(i) for i in lst]

100%|██████████| 6417/6417 [34:42<00:00,  3.08it/s]  


### JAugusto97/ToLD-Br

In [4]:
url = "https://raw.githubusercontent.com/JAugusto97/ToLD-Br/main/ToLD-BR.csv"

told_br = get_file_github(url, extension="csv")
print(f"Count: {len(told_br)}")
told_br

Count: 21000


[{'text': 'Meu nivel de amizade com isis é ela ter meu insta e eu ter o dela, e quando eu penso que não ela manda mensagem “ falano otario ta falando dnv no insta”',
  'homophobia': 0.0,
  'obscene': 0.0,
  'insult': 2.0,
  'racism': 0.0,
  'misogyny': 0.0,
  'xenophobia': 0.0},
 {'text': 'rt @user @user o cara adultera dados, que foram desmascarados e ainda quer ficar no governo?',
  'homophobia': 0.0,
  'obscene': 0.0,
  'insult': 1.0,
  'racism': 0.0,
  'misogyny': 0.0,
  'xenophobia': 0.0},
 {'text': '@user @user @user o cara só é simplesmente o maior vencedor da história de futebol, tá com 36 anos e tem gás demais e não um gordo com joelho fodido',
  'homophobia': 0.0,
  'obscene': 2.0,
  'insult': 1.0,
  'racism': 0.0,
  'misogyny': 0.0,
  'xenophobia': 0.0},
 {'text': 'eu to chorando vei vsf e eu nem staneio izone nem nada https://t.co/rglb8luutw',
  'homophobia': 0.0,
  'obscene': 1.0,
  'insult': 0.0,
  'racism': 0.0,
  'misogyny': 0.0,
  'xenophobia': 0.0},
 {'text': 'Eleitor

In [5]:
told_br = [row["text"] for row in told_br]
told_br = list(set(told_br))

print(f"Count (unique texts): {len(told_br)}")
told_br[:5]

Count (unique texts): 20813


['tu e minha porra — vai se fuder sou de ninguem não https://t.co/leoc7dadeg',
 'a vai toma no cu essa gente é chata pra carlho',
 'rt @user quero cu manauara. https://t.co/2imooikg6f',
 'E aí quem pode ir pra Brasília? Quem está próximo a Brasília. Vamos fretar ônibus, falar com empresas... Vamos invadir aquela merda .. caçar aqueles malditos... Porra o Peru conseguiu se livrar e nos não?',
 'depois diz que preto se vitimiza se foder']

In [6]:
lst = []

with tqdm(total=len(told_br)) as pbar:
    for row in told_br:
        if len(row) < 500:
            item = RawText(
                id=datetime.datetime.now().timestamp().as_integer_ratio()[0],
                text=row,
                source="ToLD-BR",
                created_at=datetime.datetime(2020, 10, 5, 14, 8, 0),
                collected_at=datetime.datetime.now()
            )

            response = perspective.predict(text=item.text)

            if isinstance(response.get("TOXICITY"), float):
                if response.get("TOXICITY") > args.PERSPECTIVE_THRESHOLD:
                    item.is_toxic = True
                    item.toxicity_score = response.get("TOXICITY")

            item.text = anonymizer.apply_all(item.text)
            
            if item.is_toxic or not args.FILTER_TOXIC_COMMENTS:
                lst.append(item)
                
        pbar.update(1)

lst = [label_studio_fmt(i) for i in lst]

100%|██████████| 20813/20813 [2:17:55<00:00,  2.51it/s]  


In [8]:
print(f"Count: {len(lst)}")
lst[0]

Count: 14046


{'data': {'text': 'tu e minha porra — vai se fuder sou de ninguem não URL',
  'ref_id': 1724648442032725,
  'meta_info': {'source': 'ToLD-BR',
   'created_at': '2020-10-05T14:08:00',
   'collected_at': '2022-02-13T08:48:43.996663',
   'is_toxic': True,
   'toxicity_score': 0.9821}}}

In [9]:
uploader = Uploader(bucket=args.AWS_S3_BUCKET,
                    bucket_prefix=args.AWS_S3_BUCKET_PREFIX)

uploader.upload_aksk(access_key=args.AWS_ACCESS_KEY_ID,
                     secret_key=args.AWS_SECRET_ACCESS_KEY,
                     key="ToLD-BR.json", data=lst)

{'ResponseMetadata': {'RequestId': 'Z0CX4A3C2B0K2H7G',
  'HostId': 'wTA4jcLKSD2XP4CUXj4ZkTF5cxUSCxfgkiSw842yJ4AVe2FqW3YbMeLaAIxSyex9yU76cvwLbYA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'wTA4jcLKSD2XP4CUXj4ZkTF5cxUSCxfgkiSw842yJ4AVe2FqW3YbMeLaAIxSyex9yU76cvwLbYA=',
   'x-amz-request-id': 'Z0CX4A3C2B0K2H7G',
   'date': 'Sun, 13 Feb 2022 14:10:32 GMT',
   'x-amz-version-id': 'I7HP0f3vc_t2zWQXiUoUMY8kcoOjvYSh',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"b70c086ad3e940be8e48dc3ccd071ab7"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"b70c086ad3e940be8e48dc3ccd071ab7"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'I7HP0f3vc_t2zWQXiUoUMY8kcoOjvYSh'}